In [1]:
"""
Pre-process dataset by extracting aligned cropped mouths. Adapted from https://github.com/mpc001/
Lipreading_using_Temporal_Convolutional_Networks/blob/master/preprocessing/crop_mouth_from_video.py"""

import argparse
import os
from collections import deque

import numpy as np
from PIL import Image
from tqdm import tqdm

from utils import warp_img, apply_transform, cut_patch

In [2]:
DATASETS = {
    "FaceForensics++": [
        "Forensics/RealFF",
        "Forensics/Deepfakes",
        "Forensics/FaceSwap",
        "Forensics/Face2Face",
        "Forensics/NeuralTextures",
    ],
    "RealFF": ["Forensics/RealFF"],
    "Deepfakes": ["Forensics/Deepfakes"],
    "FaceSwap": ["Forensics/FaceSwap"],
    "Face2Face": ["Forensics/Face2Face"],
    "NeuralTextures": ["Forensics/NeuralTextures"],
    "FaceShifter": ["Forensics/FaceShifter"],
    "DeeperForensics": ["Forensics/DeeperForensics"],
    "CelebDF": ["CelebDF/RealCelebDF", "CelebDF/FakeCelebDF"],
    "DFDC": ["DFDC"],
}
STD_SIZE = (256, 256)
STABLE_POINTS = [33, 36, 39, 42, 45]

### PREPROCESSING (LOAD THE LANDMARK WHICH IS EXTRACTED PREIVIOUSLY)

In [ ]:
import json
with open('land_fake_train.json', 'r') as f:
    land_fake =  json.load(f)
print(len(land_fake))

import json
with open('land_real_train.json', 'r') as f:
    land_real =  json.load(f)
print(len(land_real))
landmarks=dict()

for img, target in tqdm(zip(list_files, list_files_target)):
    IS_RUN = False
    if 'FAKE/' in img :
        if img in land_fake:
            landmarks[img]=land_fake[img]
    else :
        if img in land_real:
            landmarks[img]=land_real[img]


In [80]:
import tqdm
from utils import warp_img, apply_transform, cut_patch

q_landmarks = deque()
q_frames = deque()
q_name = deque()
for land in tqdm.tqdm(landmarks):
    with Image.open(land) as pil_img:
        img = np.array(pil_img)

    # Add elements to the queues
    q_frames.append(img)
    q_landmarks.append(landmarks[land])
    q_name.append(os.path.basename(land))

    if len(q_frames) == 12:  # Wait until queues are large enough
        smoothed_landmarks = np.mean(q_landmarks, axis=0)

        cur_landmarks = q_landmarks.popleft()
        cur_landmarks = np.array(cur_landmarks)
        cur_frame = q_frames.popleft()
        cur_name = q_name.popleft()

        # Get aligned frame as well as affine transformation that produced it
        trans_frame, trans = warp_img(
            smoothed_landmarks[STABLE_POINTS, :], cur_landmarks[STABLE_POINTS, :], cur_frame, STD_SIZE
        )

        # Apply that affine transform to the landmarks
        trans_landmarks = trans(cur_landmarks)

        # Crop mouth region
        cropped_frame = cut_patch(
            trans_frame,
            trans_landmarks[48 : 68],
            96 // 2,
            96 // 2,
        )

        # Save image
        target_path = land.replace('FRAMES_PNG','LIPS_PNG')
        os.makedirs(os.path.dirname(target_path),exist_ok=True)
#             target_path = os.path.join(target_dir, cur_name)
        Image.fromarray(cropped_frame.astype(np.uint8)).save(target_path)
#         break

100%|██████████| 120281/120281 [18:27<00:00, 108.62it/s]
